## SENTIMENT ANALYSIS PILPRES 2024 PADA MEDIA SOSIAL TWITTER DENGAN NAIVE BAYES CLASSIFIER

**MUCHAMMAD FAHD ISHAMUUDIN**


**41155050180048**

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import nltk
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer


In [2]:
pilpres = pd.read_csv('2022pilpres.csv')
print(pilpres.head())
print(pilpres.tail())
print(pilpres.columns)

  Unnamed: 0                       date            id  \
0          0  2022-12-30 23:50:32+00:00  1.608974e+18   
1          1  2022-12-30 23:50:16+00:00  1.608974e+18   
2          2  2022-12-30 23:40:27+00:00  1.608971e+18   
3          3  2022-12-30 23:36:07+00:00  1.608970e+18   
4          4  2022-12-30 23:26:47+00:00  1.608968e+18   

                                               tweet         username  
0  Berkat kinerja di BUMN, serta kedekatan dengan...        Fuad_Lha9  
1  @ganjarpranowo menggandeng ulama hingga tokoh ...      iwan_tata98  
2  Mantap 👍👍\nElektabilitas Menteri BUMN Erick Th...        Lima8Satu  
3  Baru Lolos Sebagai Peserta Pemilu 2024, Partai...  21BeritaTerkini  
4  Agenda politik negara tuan rumah Piala Dunia 2...      rezakaisar3  
      Unnamed: 0                       date            id  \
93877      93874  2022-01-01 03:05:00+00:00  1.477114e+18   
93878      93875  2022-01-01 02:24:46+00:00  1.477103e+18   
93879      93876  2022-01-01 02:08:17+00:0

In [3]:
len(pilpres)

93882

In [4]:
pilpres.loc[45,'tweet']

'Megawati Soekarnoputri, diyakini bakal tetap mencalonkan Ketua DPR RI Puan Maharani sebagai capres di Pilpres 2024. Puan akan melaju menjadi capres &amp; berhadapan DGN Anies Baswedan.\n\nPuan Capres PDIP\nhttps://t.co/g367x0BM1P'

In [5]:
x = 'Megawati Soekarnoputri, diyakini bakal tetap mencalonkan Ketua DPR RI Puan Maharani sebagai capres di Pilpres 2024. Puan akan melaju menjadi capres &amp; berhadapan DGN Anies Baswedan.\n\nPuan Capres PDIP\nhttps://t.co/g367x0BM1P'
print(x.lower())

megawati soekarnoputri, diyakini bakal tetap mencalonkan ketua dpr ri puan maharani sebagai capres di pilpres 2024. puan akan melaju menjadi capres &amp; berhadapan dgn anies baswedan.

puan capres pdip
https://t.co/g367x0bm1p


In [6]:
print(pilpres.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93882 entries, 0 to 93881
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  93882 non-null  object 
 1   date        93882 non-null  object 
 2   id          93879 non-null  float64
 3   tweet       93879 non-null  object 
 4   username    93876 non-null  object 
dtypes: float64(1), object(4)
memory usage: 3.6+ MB
None


In [7]:
del pilpres['Unnamed: 0']


In [8]:
print(pilpres.info())
len(pilpres)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93882 entries, 0 to 93881
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      93882 non-null  object 
 1   id        93879 non-null  float64
 2   tweet     93879 non-null  object 
 3   username  93876 non-null  object 
dtypes: float64(1), object(3)
memory usage: 2.9+ MB
None


93882

In [9]:
pilpres[pilpres['username']=='jo_anezt'].count()

date        30
id          30
tweet       30
username    30
dtype: int64

In [10]:
print(pilpres.describe(include='all'))

                             date            id  \
count                       93882  9.387900e+04   
unique                      90553           NaN   
top     2022-04-21 08:00:02+00:00           NaN   
freq                          218           NaN   
mean                          NaN  1.557322e+18   
std                           NaN  3.586852e+16   
min                           NaN  1.477076e+18   
25%                           NaN  1.529718e+18   
50%                           NaN  1.565171e+18   
75%                           NaN  1.586947e+18   
max                           NaN  1.608974e+18   

                                                    tweet username  
count                                               93879    93876  
unique                                              90724    21941  
top     Indonesia menolak Capres dan Cawapres yang did...  jpnncom  
freq                                                   48     1121  
mean                                      

In [11]:
pilpres.isna().sum()

date        0
id          3
tweet       3
username    6
dtype: int64

In [12]:
print(pilpres.username.unique())

['Fuad_Lha9' 'iwan_tata98' 'Lima8Satu' ... 'wahyu_minarno' 'Azhari25_'
 'ninovendri']


In [13]:
print(pilpres.tweet.duplicated(keep='first').sum())

3157


## CLEANING DATA
untuk melakukan sentimen analisis yang benar maka harus melakukan pembersihan data agar bisa menghindari bias

In [14]:
#karena yang null hanya ada 6 baris dari 92308 data, maka sebaiknya drop row karena hanya sekian persen dari data dan tidak memiliki pengaruh besar pada hasil sentiment nanti
pil = pilpres.dropna()
len(pil)

93876

In [15]:
#pengecekan apakah masih ada baris yang null
pil.isnull().sum()
print(pil.isnull().sum())
print(pil.head())

date        0
id          0
tweet       0
username    0
dtype: int64
                        date            id  \
0  2022-12-30 23:50:32+00:00  1.608974e+18   
1  2022-12-30 23:50:16+00:00  1.608974e+18   
2  2022-12-30 23:40:27+00:00  1.608971e+18   
3  2022-12-30 23:36:07+00:00  1.608970e+18   
4  2022-12-30 23:26:47+00:00  1.608968e+18   

                                               tweet         username  
0  Berkat kinerja di BUMN, serta kedekatan dengan...        Fuad_Lha9  
1  @ganjarpranowo menggandeng ulama hingga tokoh ...      iwan_tata98  
2  Mantap 👍👍\nElektabilitas Menteri BUMN Erick Th...        Lima8Satu  
3  Baru Lolos Sebagai Peserta Pemilu 2024, Partai...  21BeritaTerkini  
4  Agenda politik negara tuan rumah Piala Dunia 2...      rezakaisar3  


# MENGHAPUS TWEET MASSAL YANG SAMA #
agar tidak mendapatkan bias maka harus menghapus tweet yang sama yang dilakukan oleh multi account

In [16]:
pil1 = pil.drop_duplicates(subset='tweet')
print(pil1.head())
len(pil1)

                        date            id  \
0  2022-12-30 23:50:32+00:00  1.608974e+18   
1  2022-12-30 23:50:16+00:00  1.608974e+18   
2  2022-12-30 23:40:27+00:00  1.608971e+18   
3  2022-12-30 23:36:07+00:00  1.608970e+18   
4  2022-12-30 23:26:47+00:00  1.608968e+18   

                                               tweet         username  
0  Berkat kinerja di BUMN, serta kedekatan dengan...        Fuad_Lha9  
1  @ganjarpranowo menggandeng ulama hingga tokoh ...      iwan_tata98  
2  Mantap 👍👍\nElektabilitas Menteri BUMN Erick Th...        Lima8Satu  
3  Baru Lolos Sebagai Peserta Pemilu 2024, Partai...  21BeritaTerkini  
4  Agenda politik negara tuan rumah Piala Dunia 2...      rezakaisar3  


90723

In [17]:
type(pil1)
print(pil1.tweet.duplicated().sum())

0


In [18]:
import re
import string

def preprocess_tweet(tweet):
    tweet = re.sub(r'[0-9]+','', str(tweet))
    tweet = tweet.lower()
    tweet = re.sub(r'http\S+', '', tweet, flags=re.MULTILINE)
    tweet = re.sub(r'@[A-Za-z0-9_]+', '', tweet)
    tweet = re.sub(r'#[A-Za-z0-9_]+', '', tweet)
    tweet = re.sub(r'\d+', '', tweet)
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    tweet = re.sub(r'\s+', ' ', tweet)
    tweet = tweet.strip()
    return tweet

In [19]:
pilpres['tweet'] = pilpres['tweet'].apply(preprocess_tweet)

In [20]:
print(pilpres['tweet'].head())

0    berkat kinerja di bumn serta kedekatan dengan ...
1    menggandeng ulama hingga tokoh agama dalam mem...
2    mantap 👍👍 elektabilitas menteri bumn erick tho...
3    baru lolos sebagai peserta pemilu partai ummat...
4    agenda politik negara tuan rumah piala dunia j...
Name: tweet, dtype: object


In [21]:
%pip install Sastrawi

  Using cached Sastrawi-1.0.1-py2.py3-none-any.whl (209 kB)


In [22]:
!pip install nltk

In [24]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fahdmuchammad/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [27]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk import word_tokenize

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def preprocess_tweet(tweet):
    tweet = re.sub(r'[0-9]+','', str(tweet))
    tweet = tweet.lower()  # convert to lower case
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)  # remove URLs
    tweet = re.sub(r'\@\w+|\#', '', tweet)  # remove mentions and hashtags
    tweet = re.sub(r'\d+', '', tweet)  # remove numbers
    tweet = tweet.translate(str.maketrans("", "", string.punctuation))  # remove punctuations
    tweet = tweet.strip()
    return tweet

def sentiment_analysis(tweet):
    tweet = preprocess_tweet(tweet)
    tweet_tokens = word_tokenize(tweet)
    clean_tokens = [stemmer.stem(word) for word in tweet_tokens if word not in stopwords]
    positive_words = open("positive.txt").read().splitlines()
    negative_words = open("negative.txt").read().splitlines()
    positive_count = sum([1 for word in clean_tokens if word in positive_words])
    negative_count = sum([1 for word in clean_tokens if word in negative_words])
    if positive_count > negative_count:
        return 'Positive'
    elif positive_count < negative_count:
        return 'Negative'
    else:
        return 'Neutral'

pilpres['sentiment'] = pilpres['tweet'].apply(sentiment_analysis)

In [28]:
pilpres.head()

,date,id,tweet,username,sentiment
0,2022-12-30 23:50:32+00:00,1.608974e+18,berkat kinerja di bumn serta kedekatan dengan ...,Fuad_Lha9,Positive
1,2022-12-30 23:50:16+00:00,1.608974e+18,menggandeng ulama hingga tokoh agama dalam mem...,iwan_tata98,Positive
2,2022-12-30 23:40:27+00:00,1.608971e+18,mantap 👍👍 elektabilitas menteri bumn erick tho...,Lima8Satu,Neutral
3,2022-12-30 23:36:07+00:00,1.608970e+18,baru lolos sebagai peserta pemilu partai ummat...,21BeritaTerkini,Neutral
4,2022-12-30 23:26:47+00:00,1.608968e+18,agenda politik negara tuan rumah piala dunia j...,rezakaisar3,Positive


In [29]:
pilpres.to_csv('barunihh.csv', index=False)

In [30]:
pilpres.sentiment.value_counts()

Positive    46131
Neutral     33199
Negative    14552
Name: sentiment, dtype: int64

DATA TWEET Duplicate sudah dihapus